In [1]:
import numpy as np
from pynq import allocate
from pynq import Overlay
import time
import cmath

In [2]:
ol = Overlay("QFT.bit")
ol?

Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0xffffa6b8fa00>
File:            /usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay QFT.bit. The following
attributes are available on this overlay:

IP Blocks
----------
axi_stream_template_0 : pynq.overlay.DefaultIP
zynq_ultra_ps_e_0    : pynq.overlay.DefaultIP

Hierarchies
-----------
None

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
PSDDR                : Memory
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindings.

Our definition of overlay is: "post-bitstream configurable design".
Hence, this class must expose configurability through content discovery
and runtime protection.

The overlay class exposes the IP and hierarchies as attributes in the
overlay. If no other drivers a

In [3]:
qubitInput = ol.axi_stream_template_0

In [4]:
def write_qft_to_axi(overlay, state):
    """
    Write the quantum state to the AXI stream.
    """
    qubitInput = overlay.axi_stream_template_0
    
    # Ensure state is being written correctly
    q0 = np.uint32(state & 1)
    q1 = np.uint32((state >> 1) & 1)
    q2 = np.uint32((state >> 2) & 1)
    
    # Write each bit to its corresponding gateway
    qubitInput.write(0x08, int(q0))
    qubitInput.write(0x04, int(q1))
    qubitInput.write(0x00, int(q2))
    

In [5]:
def read_qft_from_axi(overlay):
    """
    Read the quantum state from the AXI stream.
    """
    qubitInput = overlay.axi_stream_template_0
    
    # Read the state of each qubit from the AXI stream
    q0 = qubitInput.read(0x0C)  # Read the state of q0 from the corresponding address
    q1 = qubitInput.read(0x10)  # Read the state of q1 from the corresponding address
    q2 = qubitInput.read(0x14)  # Read the state of q2 from the corresponding address
    
    # Return the values as a tuple (q2, q1, q0)
    return q2, q1, q0


In [6]:
# Function to compute the tensor product of three quantum states
def tensor_product_three(state1, state2, state3):
    return np.kron(np.kron(state1, state2), state3)

In [7]:
# Function to compute the amplitude of the quantum state
def compute_amplitude(qubit_states):
    """
    Compute the tensor product of the 3 qubit states and normalize the amplitude.
    """
    total_state = tensor_product_three(qubit_states[0], qubit_states[1], qubit_states[2])
    total_state /= np.linalg.norm(total_state)  # Normalise the state
    return total_state

In [8]:
def interpret_qubit_state(qubit_value, scale_factor=16):
    """
    Extracts and normalizes the real and imaginary parts of a qubit state.
    Reverses the scaling applied during transmission (default scaling factor: 16).
    Converts the extracted components to signed values if necessary.
    """
    # Extract the four 8-bit components from the 32-bit value
    alpha_real = (qubit_value >> 24) & 0xFF
    alpha_imag = (qubit_value >> 16) & 0xFF
    beta_real = (qubit_value >> 8) & 0xFF
    beta_imag = qubit_value & 0xFF
    
    # Convert from unsigned to signed 8-bit (-128 to 127 range)
    alpha_real = alpha_real - 256 if alpha_real & 0x80 else alpha_real
    alpha_imag = alpha_imag - 256 if alpha_imag & 0x80 else alpha_imag
    beta_real = beta_real - 256 if beta_real & 0x80 else beta_real
    beta_imag = beta_imag - 256 if beta_imag & 0x80 else beta_imag
    
    # Reverse the scaling
    alpha_real /= scale_factor
    alpha_imag /= scale_factor
    beta_real /= scale_factor
    beta_imag /= scale_factor
    
    # Construct complex numbers
    alpha = complex(alpha_real, alpha_imag)
    beta = complex(beta_real, beta_imag)
    
    # Normalise the quantum state properly
    norm = abs(alpha)**2 + abs(beta)**2
    if norm > 0:
        alpha /= cmath.sqrt(norm)
        beta /= cmath.sqrt(norm)
    
    # Print the quantum state for debugging
    print(f"Quantum state: |ψ> = {alpha.real:.3f}{alpha.imag:+.3f}j|0> + {beta.real:.3f}{beta.imag:+.3f}j|1>")
    
    return alpha, beta


In [9]:
# Function to run all quantum states and compute the tensor product for each |000> to |111>
def run_all_qft_inputs(overlay):
    """
    Write all possible 3-bit quantum states (|000> to |111>) to the AXI stream
    and read the results, then compute the amplitudes for each state.
    """
    state_amplitudes = {}  # Dictionary to store computed states

    for state in range(8):  # Loop through all possible 3-qubit states
        print(f"\nRunning state |{state:03b}> (decimal {state})")
        
        # Write the quantum state to the AXI stream
        write_qft_to_axi(overlay, state)
        
        # Add a small delay to allow processing
        time.sleep(0.1)  
        
        # Read the output from the AXI stream
        result = read_qft_from_axi(overlay)
        
        print(result)
        
        # Extract qubit states
        qubit_states = []
        for qubit_value in result:
            alpha, beta = interpret_qubit_state(qubit_value)
            qubit_states.append(np.array([alpha, beta], dtype=complex))

        # Compute the tensor product of the qubit states
        amplitude = compute_amplitude(qubit_states)
        
        # Store the final quantum state
        state_amplitudes[state] = amplitude

    return state_amplitudes

# Run the function
state_amplitudes = run_all_qft_inputs(ol)



Running state |000> (decimal 0)
(754986240, 754986240, 754986240)
Quantum state: |ψ> = 0.707+0.000j|0> + 0.707+0.000j|1>
Quantum state: |ψ> = 0.707+0.000j|0> + 0.707+0.000j|1>
Quantum state: |ψ> = 0.707+0.000j|0> + 0.707+0.000j|1>

Running state |001> (decimal 1)
(755028480, 754986240, 754986240)
Quantum state: |ψ> = 0.699+0.000j|0> + -0.715+0.000j|1>
Quantum state: |ψ> = 0.707+0.000j|0> + 0.707+0.000j|1>
Quantum state: |ψ> = 0.707+0.000j|0> + 0.707+0.000j|1>

Running state |010> (decimal 2)
(754974765, 755028480, 754986240)
Quantum state: |ψ> = 0.707+0.000j|0> + 0.000+0.707j|1>
Quantum state: |ψ> = 0.699+0.000j|0> + -0.715+0.000j|1>
Quantum state: |ψ> = 0.707+0.000j|0> + 0.707+0.000j|1>

Running state |011> (decimal 3)
(754974930, 755028480, 754986240)
Quantum state: |ψ> = 0.699+0.000j|0> + 0.000-0.715j|1>
Quantum state: |ψ> = 0.699+0.000j|0> + -0.715+0.000j|1>
Quantum state: |ψ> = 0.707+0.000j|0> + 0.707+0.000j|1>

Running state |100> (decimal 4)
(754982687, 754974765, 755028480)
Qu

In [10]:
# 2D array to store the quantum state vectors for each state (|000> to |111>)
qubit_states = np.zeros((8, 3, 2), dtype=complex)

# Extract FPGA QFT output
qubit_states = {}

# Running state |000>
qubit_states[0] = [np.array([0.707+0.000j, 0.707+0.000j]),
                   np.array([0.707+0.000j, 0.707+0.000j]),
                   np.array([0.707+0.000j, 0.707+0.000j])]

# Running state |001>
qubit_states[1] = [np.array([0.699+0.000j, -0.715+0.000j]),
                   np.array([0.707+0.000j, 0.707+0.000j]),
                   np.array([0.707+0.000j, 0.707+0.000j])]

# Running state |010>
qubit_states[2] = [np.array([0.707+0.000j, 0.000+0.707j]),
                   np.array([0.699+0.000j, -0.715+0.000j]),
                   np.array([0.707+0.000j, 0.707+0.000j])]

# Running state |011>
qubit_states[3] = [np.array([0.699+0.000j, 0.000-0.715j]),
                   np.array([0.699+0.000j, -0.715+0.000j]),
                   np.array([0.707+0.000j, 0.707+0.000j])]

# Running state |100>
qubit_states[4] = [np.array([0.716+0.000j, 0.493+0.493j]),
                   np.array([0.707+0.000j, 0.000+0.707j]),
                   np.array([0.699+0.000j, -0.715+0.000j])]

# Running state |101>
qubit_states[5] = [np.array([0.694+0.000j, -0.509-0.509j]),
                   np.array([0.707+0.000j, 0.000+0.707j]),
                   np.array([0.699+0.000j, -0.715+0.000j])]

# Running state |110>
qubit_states[6] = [np.array([0.711+0.000j, -0.505+0.490j]),
                   np.array([0.699+0.000j, 0.000-0.715j]),
                   np.array([0.699+0.000j, -0.715+0.000j])]

# Running state |111>
qubit_states[7] = [np.array([0.700+0.000j, 0.497-0.513j]),
                   np.array([0.699+0.000j, 0.000-0.715j]),
                   np.array([0.699+0.000j, -0.715+0.000j])]

# Compute and display the tensor product only for the corresponding input state
for state_index in range(8):
    q1, q2, q3 = qubit_states[state_index]
    quantum_state = tensor_product_three(q1, q2, q3).flatten()
    
for i in range(8):
    print(f"|{i:03b}> -> {quantum_state[i]}")


|000> -> (0.34202069999999996+0j)
|001> -> (-0.3498495+0j)
|010> -> -0.3498494999999999j
|011> -> 0.35785749999999994j
|100> -> (0.24283469699999996-0.25065231299999996j)
|101> -> (-0.24839314499999995+0.25638970499999997j)
|110> -> (-0.25638970499999997-0.24839314499999995j)
|111> -> (0.26225842499999996+0.25407882499999995j)


In [11]:
# Qiskit expected outputs
qiskit_output = np.array([
    0.3535533905932737+0j,
    -0.3535533905932737+0j,
    4.6909283434219394e-07+0.3535533905929625j,
    -4.6909283434219394e-07-0.3535533905929625j,
    -0.250000165849307-0.24999983415058283j,
    0.250000165849307+0.24999983415058283j,
    0.24999950245141855-0.25000049754759107j,
    -0.24999950245141855+0.25000049754759107j
])

# PL results
fpga_output = np.array([
    0.34202069999999996+0j,
    -0.3498495+0j,
    -0.3498494999999999j,
    0.35785749999999994j,
    0.24283469699999996-0.25065231299999996j,
    -0.24839314499999995+0.25638970499999997j,
    -0.25638970499999997-0.24839314499999995j,
    0.26225842499999996+0.25407882499999995j
])

def normalise(state):
    return state / np.linalg.norm(state)

fpga_output = normalise(fpga_output)
qiskit_output = normalise(qiskit_output)

# Compute relative phases
def relative_phases(state):
    return np.angle(state[1:] / state[:-1])

fpga_phases = relative_phases(fpga_output)
qiskit_phases = relative_phases(qiskit_output)

print("FPGA Relative Phases:", fpga_phases)
print("Qiskit Relative Phases:", qiskit_phases)

phase_diff = np.angle(np.exp(1j * (fpga_phases - qiskit_phases)))

print("\nRelative Phase Differences (radians):")
for i, diff in enumerate(phase_diff):
    print(f"|{i:03b}> -> |{i+1:03b}>: {diff:.5f} rad")


FPGA Relative Phases: [ 3.14159265  1.57079633 -3.14159265 -2.37203475 -3.14159265  1.57079633
 -3.14159265]
Qiskit Relative Phases: [ 3.14159265 -1.57079765  3.14159265 -0.7853975  -3.14159265 -1.57079765
 -3.14159265]

Relative Phase Differences (radians):
|000> -> |001>: 0.00000 rad
|001> -> |010>: -3.14159 rad
|010> -> |011>: 0.00000 rad
|011> -> |100>: -1.58664 rad
|100> -> |101>: 0.00000 rad
|101> -> |110>: -3.14159 rad
|110> -> |111>: 0.00000 rad
